In [1]:
import numpy as np
import pandas as pd
import gym
from algorithms import QLearning

# 1. Taxi environment

In [2]:
""" Environment information
ref: https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py#L24

Actions
    There are 6 discrete deterministic actions:
    - 0: move south
    - 1: move north
    - 2: move east
    - 3: move west
    - 4: pickup passenger
    - 5: drop off passenger
Observations
    There are 500 discrete states since there are 25 taxi positions, 5 possible
    locations of the passenger (including the case when the passenger is in the
    taxi), and 4 destination locations.
    Note that there are 400 states that can actually be reached during an
    episode. The missing states correspond to situations in which the passenger
    is at the same location as their destination, as this typically signals the
    end of an episode. Four additional states can be observed right after a
    successful episodes, when both the passenger and the taxi are at the destination.
    This gives a total of 404 reachable discrete states.
    Each state space is represented by the tuple:
    (taxi_row, taxi_col, passenger_location, destination)
    An observation is an integer that encodes the corresponding state.
    The state tuple can then be decoded with the "decode" method.
    Passenger locations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
    - 4: in taxi
    Destinations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
Info
    ``step`` and ``reset()`` will return an info dictionary that contains "p" and "action_mask" containing
        the probability that the state is taken and a mask of what actions will result in a change of state to speed up training.
    As Taxi's initial state is a stochastic, the "p" key represents the probability of the
    transition however this value is currently bugged being 1.0, this will be fixed soon.
    As the steps are deterministic, "p" represents the probability of the transition which is always 1.0
    For some cases, taking an action will have no effect on the state of the agent.
    In v0.25.0, ``info["action_mask"]`` contains a np.ndarray for each of the action specifying
    if the action will change the state.
    To sample a modifying action, use ``action = env.action_space.sample(info["action_mask"])``
    Or with a Q-value based algorithm ``action = np.argmax(q_values[obs, np.where(info["action_mask"] == 1)[0]])``.
Rewards
    - -1 per step unless other reward is triggered.
    - +20 delivering passenger.
    - -10  executing "pickup" and "drop-off" actions illegally.
"""
env = gym.make("Taxi-v3")

In [3]:
state = env.reset()
taxi_row, taxi_col, passenger_location, destination = env.decode(state)
print(taxi_row, taxi_col, passenger_location, destination)
env.render()

3 4 0 1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



# 2. Q-Learning

In [4]:
class QLearningTaxi(QLearning):
    def __init__(self, env, alpha=0.1, gamma=0.99, epsilon=0.1):
        super().__init__(env, alpha, gamma, epsilon)
        
        # CUSTOMIZE: the environment shape for Taxi-v3 is (5, 5)
        self.env_shape = (5, 5)

    # TODO: add display option for visualization of the episode
    def generate_sample_episode(self, display=False, always_greedy=False):
        episode = list()
    
        state = self.env.reset()
        while True:
            timestep = list()
            prev_state = state
            
            action = self.choose_action(state, always_greedy=always_greedy)
            state, reward, done, _ = self.env.step(action)
            
            timestep.append(prev_state)
            timestep.append(action)
            timestep.append(reward)
            episode.append(timestep)

            if done:
                # include the terminal state
                episode.append([state, None, None])
                break
        
        return episode

    # CUSTOMIZE: for Taxi environment
    def evaluate_policy(self, episodes, display=False):
        wins = 0
        for _ in range(episodes):
            episode = self.generate_sample_episode(display=False)
            if episode[-2][-1] == 20:
                wins += 1
        return wins / episodes

In [5]:
env = gym.make("Taxi-v3")
qlearning_agent = QLearningTaxi(env)
qlearning_agent.train(episodes=3000)

In [6]:
# test the trained agent
test_episodes = 100
win_rate = qlearning_agent.evaluate_policy(episodes=test_episodes)
print(f'agent win rate: {win_rate*100 :.2f}% from {test_episodes} test episodes')

agent win rate: 94.00% from 100 test episodes


In [7]:
print("Q-table shape: ", qlearning_agent.Q.shape)
qlearning_agent.Q

Q-table shape:  (500, 6)


array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-13.72768255, -13.74479488, -13.75266806, -13.99915753,
        -12.61982832, -15.72226812],
       [ -9.16139051,  -8.7947854 ,  -9.18620862,  -9.67581796,
         -6.90165854,  -9.23304457],
       ...,
       [ -9.68684327,  -8.65593796,  -9.29741165,  -9.36345029,
         -9.7239857 ,  -9.47741436],
       [-15.07525683, -14.39023442, -14.71135515, -15.04737902,
        -15.13193599, -15.87478191],
       [ -5.3518972 ,  -4.57890024,  -5.26697884,  -2.44609066,
         -5.3862768 ,  -4.77487501]])

In [8]:
# sample episode (state, action, reward)
qlearning_agent.generate_sample_episode()

[[481, 3, -1],
 [461, 0, -1],
 [461, 1, -1],
 [361, 1, -1],
 [261, 3, -1],
 [241, 3, -1],
 [221, 3, -1],
 [201, 1, -1],
 [101, 1, -1],
 [1, 4, -1],
 [17, 2, -1],
 [37, 0, -1],
 [137, 0, -1],
 [237, 2, -1],
 [257, 2, -1],
 [277, 5, -10],
 [277, 0, -1],
 [377, 1, -1],
 [277, 1, -1],
 [177, 1, -1],
 [77, 2, -1],
 [97, 5, 20],
 [85, None, None]]

# 3. Double Q-Learning

In [9]:
class DoubleQLearning(QLearningTaxi):
    def __init__(self, env, alpha=0.1, gamma=0.99, epsilon=0.1):
        super().__init__(env, alpha, gamma, epsilon)

        self.Q = None
        self.Q1 = np.zeros((env.observation_space.n, env.action_space.n))
        self.Q2 = np.zeros((env.observation_space.n, env.action_space.n))

    def choose_action(self, state, always_greedy=False):
        if always_greedy:
            return np.argmax(self.Q1[state] + self.Q2[state])
        if np.random.rand() < self.epsilon:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.Q1[state] + self.Q2[state])

    def update_Q(self, states, actions, rewards):
        """
        Updates the Q-table using the double Q-learning update rule
        """
        state, action, reward = states[0], actions[0], rewards[0]
        next_state = states[-1]
        
        if np.random.rand() < 0.5:
            a_best = np.argmax(self.Q1[next_state])
            self.Q1[state, action] += self.alpha * (reward + self.gamma * self.Q2[next_state, a_best] - self.Q1[state, action])
        else:
            a_best = np.argmax(self.Q2[next_state])
            self.Q2[state, action] += self.alpha * (reward + self.gamma * self.Q1[next_state, a_best] - self.Q2[state, action])

In [10]:
env = gym.make("Taxi-v3")
dqlearning_agent = DoubleQLearning(env)
dqlearning_agent.train(episodes=4000)

In [11]:
# test the trained agent
test_episodes = 100
win_rate = dqlearning_agent.evaluate_policy(episodes=test_episodes)
print(f'agent win rate: {win_rate*100 :.2f}% from {test_episodes} test episodes')

agent win rate: 99.00% from 100 test episodes


In [12]:
print("Q-table 1 shape: ", dqlearning_agent.Q1.shape)
dqlearning_agent.Q1

Q-table 1 shape:  (500, 6)


array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-20.75542918, -19.92252507, -19.09488525, -20.33213634,
        -13.29708345, -19.84209317],
       [-11.50147523, -12.16690577, -12.5123893 , -10.98573195,
         -6.66240232, -10.6336982 ],
       ...,
       [ -8.06850751,  -5.98364111,  -7.77893965,  -8.7499446 ,
         -8.96907139,  -7.97492356],
       [-16.37555612, -15.46464557, -16.13693798, -15.93362815,
        -17.08503729, -15.6740936 ],
       [ -3.53572206,  -3.84330201,  -3.25997173,  -2.24443221,
         -3.57953828,  -2.77487504]])

In [13]:
print("Q-table 2 shape: ", dqlearning_agent.Q2.shape)
dqlearning_agent.Q2

Q-table 2 shape:  (500, 6)


array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [-19.54806922, -21.23248661, -21.9285479 , -19.88538524,
        -13.03468397, -21.2986652 ],
       [-13.22547207, -11.53844687, -12.71131201, -12.18916618,
         -7.33613326, -13.13661204],
       ...,
       [ -8.19410617,  -6.39092793,  -9.25827374,  -7.92731667,
         -7.54083315,  -8.34796327],
       [-16.32042095, -17.30005405, -17.31658263, -17.21567391,
        -16.59577138, -18.08610177],
       [ -2.83890115,  -3.25159896,  -3.13839664,  -2.63395011,
         -2.98031826,  -3.44827867]])

In [14]:
# Q1 and Q2 average value difference
np.mean(dqlearning_agent.Q1 - dqlearning_agent.Q2)

-0.012514196510188183

In [16]:
# sample episode (state, action, reward)
dqlearning_agent.generate_sample_episode()

[[329, 1, -1],
 [229, 3, -1],
 [209, 0, -1],
 [309, 0, -1],
 [409, 0, -1],
 [409, 4, -1],
 [417, 1, -1],
 [317, 1, -1],
 [217, 2, -1],
 [237, 2, -1],
 [257, 1, -1],
 [157, 1, -1],
 [57, 2, -1],
 [77, 2, -1],
 [97, 5, 20],
 [85, None, None]]